# Week 4
## Introduction
### Divide and Conquer Algorithm
**Divide**: Break into **non-overlapping** subproblems of the **same type**.

**Conquer**: solve subproblems and combine.

Because each subproblem is the same type as our original problem, it is naturally to solve the problems recursively. 

### Linear Search in Array
**Input**: An array $A$ with $n$ elements. A key $k$.

**Output**: An index, $i$, where $A[i]=k$. If there is no such $i$, then NOT_FOUND.

Below is the recursive version.

In [1]:
def LinearSearch(A, low, high, key):
    if high < low:
        return NOT_FOUND
    if A[low] == key:
        return low
    return LinearSearch(A, low+1, high, key)

A <span style="color:blue">recurrence relation</span> is an equation recursively defining a sequence of values. Example: Fibonacci numbers.

Recurrence defining worst-case time: $T(n) = T(n-1) + c$ and $T(0) = c$. Thus, the worse-case runtime for LinearSearch is $O(n)$.

The iterative version of LinearSearch is:

In [2]:
def LinearSearch(A, low, high, key):
    for i in range(low, high):
        if A[i] == key:
            return i
    return NOT_FOUND

### Binary Search
**Input**: A sorted array $A$, elements in $A$ can repeat, and a key $k$

**Output**: An index, $i$, where $A[i] = k$; Otherwise, the greatest index $i$, where $A[i] < k$; Otherwise, *low-1*.

In [3]:
import math

def BinarySearch(A, low, high, key):
    if high < low:
        return low - 1
    mid = math.floor(low + (high - low)/2)
    if key == A[mid]:
        return mid
    else if key < A[mid]:
        return BinarySearch(A, low, mid - 1, key)
    else if key > A[mid]:
        return BinarySearch(A, mid + 1, high, key)

SyntaxError: invalid syntax (<ipython-input-3-093b4ae49204>, line 7)

#### Binary Search Runtime Analysis

Binary search recurrence relation: $T(n) = T\left(\left\lfloor n/2 \right\rfloor\right) + c$, $T(0) = c$. There will be $log_2n$ times for an array of length $n$. So we could sum the time up and the total runtime is $O(log_2n)$. It can be written as $O(\log n)$ as the base is not important.

**Iterative Version**

In [1]:
import math

def BinarySearchIt(A, low, high, key):
    while low <= high:
        mid = math.floor(low + (high - low) / 2)
        if key == A[mid]:
            return mid
        else if key < A[mid]:
            high = mid - 1
        else:
            low = mid + 1
    return low - 1

SyntaxError: invalid syntax (<ipython-input-1-b3d48ec4622c>, line 6)

## Polynomial Multiplication
Input: Two $n - 1$ degree polynomials: $A = (a_{n-1}, a_{n-2}, ..., a_1, a_0)$ and $B = (b_{n-1}, b_{n-2}, ..., b_1, b_0)$
\begin{align*}
&a_{n-1}x^{n-1} + a_{n-2}x^{n-2} + ... + a_1x + a_0 \\
&b_{n-1}x^{n-1} + b_{n-2}x^{n-2} + ... + b_1x + b_0
\end{align*}
Output: The product polynomial:
\begin{equation*}
c_{2n-2}x^{2n-2} + c_{2n-3}x^{2n-3} + ... + c_1x + c_0
\end{equation*}
where $c_{2n-2} = a_{n-1}b_{n-1}$, $c_{2n-3} = a_{n-1}b_{n-2} + a_{n-2}b_{n-1}$, ..., $c_1 = a_1b_0+a_0b_1$, $c_0=a_0b_0$.

Thus, we could see the naive algorithm is

In [2]:
import numpy as np

def MultPoly(A, B, n):
    product = np.zeros(2*n-1)
    for i in range(n-1):
        for j in range(n-1):
            product[i+j] += A[i]*B[j]
    return product

The runtime for `MultPoly()` function is $O(n^2)$.

### Naive Divide and Conquer Algorithm
Let $A(x) = D_1(x)x^{\frac{n}{2}} + D_0(x)$ where
\begin{equation*}
    D_1(x) = a_{n-1}x^{\frac{n}{2}-1} + a_{n-2}x^{\frac{n}{2}-2} + ... + a_{\frac{n}{2}} \\
    D_0(x) = a_{\frac{n}{2}-1}x^{\frac{n}{2}-1} + a_{\frac{n}{2}-2}x^{\frac{n}{2}-2} + ... + a_0
\end{equation*}
and $B(x) = E_1(x)x^{\frac{n}{2}} + E_0(x)$ where $E_1(x)$ and $E_0(x)$ have similar form as $D(x)$.

Then, we have 
\begin{align*}
    AB &= (D_1x^{\frac{n}{2}} + D_0)(E_1x^{\frac{n}{2}} + E_0) \\
       &= (D_1E_1)x^n + (D_1E_0 + D_0E_1)x^{\frac{n}{2}} + D_0E_0
\end{align*}

Recurrence: the runtime is $T(n)=4T(\frac{n}{2})+kn$, i.e. $T(n)=4T(\frac{n}{2})+O(n)$.

In [7]:
import numpy as np

def Mult2(A, B, n, ai, bi):
    """
    ai, bi: coefficients that we are interested in.
    """
    R = np.zeros(2*n-1)
    if n == 1:
        R[0] = A[ai] * B[bi]
        return R
    R[0:n-2] = Mult2(A, B, n/2, ai, bi)
    R[n:2*n-2] = Mult2(A, B, n/2, ai+n/2, bi+n/2)
    D0E1 = Mult2(A, B, n/2, ai, bi+n/2)
    D1E0 = Mult2(A, B, n/2, ai+n/2, bi)
    R[n/2:n+n/2-2] += D1E0 + D0E1
    return R

`Mult2()` function requires that $n$ is a power of 2 so the code can safely calculate $\frac{n}{2}$ without worrying about rounding. It can be achieved by padding each polynomial with zero terms to the needed degree.

The runtime is $\sum_{i=0}^{log_2n}4^ik\frac{n}{2^i} = O(n^2)$, which is the same as the naive algorithm.

### Karatsuba approach
Rewrite $C(x) = a_1b_1x^2 + (a_1b_0 + a_0b_1)x + a_0b_0$ as 
\begin{equation*}
    C(x) = a_1b_1x^2 + ((a_1 + a_0)(b_1 + b_0) - a_1b_1 - a_0b_0)x + a_0b_0
\end{equation*}
which only needs 3 multiplications.

The runtime is $\sum_{i=0}^{\log_2n}k\frac{n}{2^i}=O(n^{\log_23})=O(n^{1.58})$. Note: the extra additions change the $k$ constant here.

## Master Theorem

If $T(n) = aT(\lceil \frac{n}{b} \rceil) + O(n^d)$ (for constants $a > 0$, $b > 0$, $d \geq 0$), then:

\begin{equation*}
    T(n) = 
    \begin{cases}
        O(n^d) & \text{if $d>\log_ba$} \\
        O(n^d\log n) & \text{if $d = \log_ba$}\\
        O(n^{\log_ba}) & \text{if $d <  \log_ba$}\\
    \end{cases}
\end{equation*}

To prove the above therom, we first can derive that runtime is $\sum_{i=0}^{\log_bn}O(n^d)\left(\frac{a}{b^d}\right)^i$. Then based upon geometric series below, there should be three cases.

### Geometric Series
For $r \neq 1$:

\begin{align*}
    &a + ar + ar^2 + ar^3 + ... + ar^{n - 1} \\
    = &a \frac{1 - r ^n}{1 - r} \\
    = & \begin{cases}
        O(a) & \text{if $r < 1$} \\
        O(ar^{n-1}) & \text{if $r > 1$}
    \end{cases}
\end{align*}

Note: $a^{log_bn} = (b^{log_ba})^{log_bn} = (b^{log_bn})^{log_ba} = n^{log_ba}$.

## Sorting Problem

**Input**: Sequence $A[1...n]$

**Output**: Permutation $A'[1...n]$ of $A[1...n]$ in non-decreasing order.

### Selection Sort
* Find a minimum by scanning the array
* Swap it with the first element
* Repeat with the remaining part of the array

In [8]:
def swap(Ai, Aj):
    temp = Ai
    Ai = Aj
    Aj = temp
    return Ai, Aj

def SelectionSort(A):
    n = len(A)
    for i in range(n):
        minIndex = i
        for j in range(i+1, n):
            if A[j] < A[minIndex]:
                minIndex = j
        swap(A[i], A[minIndex])

The running time of `SelectionSort()` is $O(n^2)$. Proof: $n$ iterations of outer loop, at most $n$ iterations of inner loop.

Sort in place: requires a constant amount of extra memory.

### Merge Sort
* split the array into two halves
* sort the halves recursively
* merge the sorted halves into one array

In [9]:
import numpy as np
import math

def Merge(B, C):
    '''
    B: a numpy array of size len(B)
    C: a numpy array of size len(C)
    '''
    D = np.zeros(len(B) + len(C))
    i, j, k = 0, 0, 0
    while (i < len(B)) and (j < len(C)):
        if B[i] <= C[j]:
            D[k] = B[i]
            i += 1
        else:
            D[k] = C[j]
            j += 1
        k += 1
    while i < len(B):
        D[k] = B[i]
        k += 1
        i += 1
    while j < len(C):
        D[k] = C[j]
        k += 1
        j += 1
    return D

def MergeSort(A):
    '''
    A: a numpy array of size n
    '''
    n = len(A)
    if n == 1:
        return A
    m = math.floor(n / 2)
    B = MergeSort(A[:m])
    C = MergeSort(A[m:])
    new_A = Merge(B, C)
    return new_A    

The running time of `MergeSort()` is $O(n\log n)$.

Proof:
* The running time of merging $B$ and $C$ is $O(n)$.
* It satisties a recurrence $T(n) \leq 2T(n/2) + O(n)$.

### Lower Bound for Comparison Based Sorting
* A comparison based sorting algortihm sorts objects by comparing pairs of them.

Any comparison based sorting algorithm performs $\Omega(n\log n)$ comparisons in the worst case to sort $n$ objects.

The number of leaves $\ell$ in the comparison tree must be at least $n!$ (the total number of permutations).

The worst-case running time of the algorithm (the number of comparisons made) is at least the depth $d$, where $d \geq \log_2{\ell}$.

\begin{equation*}
    \begin{split}
        \log_2(n!) &= \log_2(1 \cdot2 \cdots n) \\
                  &= \log_21 + \log_22 + \cdots + \log_2n \\
                  &\geq \log_2\frac{n}{2} + \cdots + \log_2n   \text{(throw away first half)}\\ 
                  &\geq \frac{n}{2}\log_2\frac{n}{2} = \Omega(n\log n) \\
    \end{split}
\end{equation*}

### Non-Comparison Based Sorting Algorithm

#### Counting Sort
* Assume that all elements of $A[1...n]$ are integers from $1$ to $M$.
* By a single scan of the array $A$, count the number of occurrences of each $1 \leq k \leq M$ in the array $A$ and store it in $Count[k]$.
* Using this information, fill in the sorted array $A'$.

In [55]:
def CountSort(A):
    '''
    A: an array of size n
    m: m different numbers in array A
    '''
    n = len(A)
    count = {}
    for i in range(n):
        if A[i] not in count:
            count[A[i]] = 1
        else:
            count[A[i]] += 1
    new_A = ["" for i in range(n)]
    k = 0
    for i in list(count.keys()):
        for j in range(count[i]):
            new_A[k] = i
            k += 1

    return new_A

Provided that all elements of $A$ are integers from 1 to $M$, `CountSort()` sorts $A$ in time $O(n+M)$.